### Loading the necessary libraries:

In [1]:
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn import tree, ensemble
from sklearn.linear_model import LinearRegression

In [2]:
with open('flights_after_wrangling.pkl', 'rb') as file:
    df = pickle.load(file)

### Examining the data:

In [3]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ORIG_STATE,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_LATITUDE,DEST_LONGITUDE,DATE,DELAY_LEVEL
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,...,AK,61.17432,-149.99619,Seattle-Tacoma International Airport,Seattle,WA,47.44898,-122.30931,2015-01-01,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,...,CA,33.94254,-118.40807,Palm Beach International Airport,West Palm Beach,FL,26.68316,-80.09559,2015-01-01,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,...,CA,37.61900,-122.37484,Charlotte Douglas International Airport,Charlotte,NC,35.21401,-80.94313,2015-01-01,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,...,CA,33.94254,-118.40807,Miami International Airport,Miami,FL,25.79325,-80.29056,2015-01-01,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,...,WA,47.44898,-122.30931,Ted Stevens Anchorage International Airport,Anchorage,AK,61.17432,-149.99619,2015-01-01,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1403471 entries, 0 to 1403470
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   YEAR                 1403471 non-null  int64         
 1   MONTH                1403471 non-null  int64         
 2   DAY                  1403471 non-null  int64         
 3   DAY_OF_WEEK          1403471 non-null  int64         
 4   AIRLINE_CODE         1403471 non-null  object        
 5   FLIGHT_NUMBER        1403471 non-null  int64         
 6   TAIL_NUMBER          1395281 non-null  object        
 7   ORIGIN_AIRPORT       1403471 non-null  object        
 8   DESTINATION_AIRPORT  1403471 non-null  object        
 9   SCHEDULED_DEPARTURE  1403471 non-null  datetime64[ns]
 10  DEPARTURE_DELAY      1361116 non-null  float64       
 11  TAXI_OUT             1360300 non-null  float64       
 12  WHEELS_OFF           1360300 non-null  float64       
 1

My goal is to build a model that is accurately able to predict, given certain features, whether a flight is going to be delayed or not. In order to do so, I want to determine which of these features are useful, i.e., which of these features contain information that is useful to predict delay. 

In [5]:
df[df.columns[1:]].corr()['ARRIVAL_DELAY'][:].sort_values(ascending=False)

ARRIVAL_DELAY          1.000000
DEPARTURE_DELAY        0.940434
DELAY_LEVEL            0.684484
AIRLINE_DELAY          0.600503
LATE_AIRCRAFT_DELAY    0.522679
WEATHER_DELAY          0.314062
TAXI_OUT               0.268260
AIR_SYSTEM_DELAY       0.243839
WHEELS_OFF             0.153788
TAXI_IN                0.140882
WHEELS_ON              0.065830
ELAPSED_TIME           0.040286
DEST_LONGITUDE         0.039928
ORIG_LONGITUDE         0.027617
FLIGHT_NUMBER          0.026261
DEST_LATITUDE          0.014206
SECURITY_DELAY         0.012062
ORIG_LATITUDE          0.009910
DAY_OF_WEEK           -0.003578
AIR_TIME              -0.006497
MONTH                 -0.009887
DISTANCE              -0.025270
SCHEDULED_TIME        -0.029482
DAY                   -0.079890
DIVERTED                    NaN
CANCELLED                   NaN
Name: ARRIVAL_DELAY, dtype: float64

In [6]:
df['DAY_OF_WEEK'].unique()

array([4, 5, 6, 7, 1, 2, 3])

Upon examining the documentation of the data in Kaggle, the way that this feature is defined is by using Monday as 1, Tuesday as 2, and so on.

I want to create another column which will be a binary indicator of whether the flight departed on a weekend or a weekday.

In [7]:
df['WEEKDAY'] = df['DAY_OF_WEEK'].apply(lambda x: 1 if x in [1,2,3,4,5] else 0)

In [8]:
df['DELAY_LEVEL'].unique()

array([0, 1, 2])

Now, I am going to create a subset of the data with only the features that I will use for modeling and predicting arrival delay.

In [9]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_CODE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON',
       'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'DIVERTED',
       'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY', 'AIRLINE', 'ORIG_AIRPORT', 'ORIG_CITY', 'ORIG_STATE',
       'ORIG_LATITUDE', 'ORIG_LONGITUDE', 'DEST_AIRPORT', 'DEST_CITY',
       'DEST_STATE', 'DEST_LATITUDE', 'DEST_LONGITUDE', 'DATE', 'DELAY_LEVEL',
       'WEEKDAY'],
      dtype='object')

In [10]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_LATITUDE,DEST_LONGITUDE,DATE,DELAY_LEVEL,WEEKDAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,...,61.17432,-149.99619,Seattle-Tacoma International Airport,Seattle,WA,47.44898,-122.30931,2015-01-01,0,1
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,...,33.94254,-118.40807,Palm Beach International Airport,West Palm Beach,FL,26.68316,-80.09559,2015-01-01,0,1
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,...,37.61900,-122.37484,Charlotte Douglas International Airport,Charlotte,NC,35.21401,-80.94313,2015-01-01,0,1
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,...,33.94254,-118.40807,Miami International Airport,Miami,FL,25.79325,-80.29056,2015-01-01,0,1
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,...,47.44898,-122.30931,Ted Stevens Anchorage International Airport,Anchorage,AK,61.17432,-149.99619,2015-01-01,0,1


In [11]:
df.drop(['YEAR', 'DAY', 'TAIL_NUMBER', 'FLIGHT_NUMBER', 
              'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'WHEELS_OFF',
              'WHEELS_ON','CANCELLATION_REASON', 'AIRLINE',
              'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ORIG_LATITUDE',
              'ORIG_CITY','ORIG_STATE','ORIG_LONGITUDE', 'DEST_CITY', 
              'DEST_LATITUDE', 'DEST_LONGITUDE', 'DATE', 'DELAY_LEVEL', 'DEST_STATE'], axis=1, inplace=True)

In [12]:
df.columns

Index(['MONTH', 'DAY_OF_WEEK', 'AIRLINE_CODE', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY', 'ORIG_AIRPORT', 'DEST_AIRPORT', 'WEEKDAY'],
      dtype='object')

In [13]:
df.head()

,MONTH,DAY_OF_WEEK,AIRLINE_CODE,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,TAXI_IN,...,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,ORIG_AIRPORT,DEST_AIRPORT,WEEKDAY
0,1,4,AS,-11.0,21.0,205.0,194.0,169.0,1448,4.0,...,0,0,NaN,NaN,NaN,NaN,NaN,Ted Stevens Anchorage International Airport,Seattle-Tacoma International Airport,1
1,1,4,AA,-8.0,12.0,280.0,279.0,263.0,2330,4.0,...,0,0,NaN,NaN,NaN,NaN,NaN,Los Angeles International Airport,Palm Beach International Airport,1
2,1,4,US,-2.0,16.0,286.0,293.0,266.0,2296,11.0,...,0,0,NaN,NaN,NaN,NaN,NaN,San Francisco International Airport,Charlotte Douglas International Airport,1
3,1,4,AA,-5.0,15.0,285.0,281.0,258.0,2342,8.0,...,0,0,NaN,NaN,NaN,NaN,NaN,Los Angeles International Airport,Miami International Airport,1
4,1,4,AS,-1.0,11.0,235.0,215.0,199.0,1448,5.0,...,0,0,NaN,NaN,NaN,NaN,NaN,Seattle-Tacoma International Airport,Ted Stevens Anchorage International Airport,1


I am going to impute the missing values in the dataset using the median of the respective columns since the median is a much more robust measure than the mean.

In [14]:
df = df.fillna(df.median())

I am going to examine how many flights have a large arrival delay. I'm defining a large delay as any delay above 15 minutes.

In [15]:
delay = []

for val in df['ARRIVAL_DELAY']:
    if val > 15:
        delay.append(1)
    else:
        delay.append(0)  

df['DELAY'] = delay

In [16]:
df.value_counts('DELAY')

DELAY
0    1127523
1     275948
dtype: int64

I see that there are more flights with a <= 15 min arrival delay than flights with a larger arrival delay.

Now, it's time to move onto the delay prediction!

In [17]:
X = df.drop(['DELAY', 'ARRIVAL_DELAY'], axis=1)
y = df['ARRIVAL_DELAY']

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1403471 entries, 0 to 1403470
Data columns (total 20 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   MONTH                1403471 non-null  int64  
 1   DAY_OF_WEEK          1403471 non-null  int64  
 2   AIRLINE_CODE         1403471 non-null  object 
 3   DEPARTURE_DELAY      1403471 non-null  float64
 4   TAXI_OUT             1403471 non-null  float64
 5   SCHEDULED_TIME       1403471 non-null  float64
 6   ELAPSED_TIME         1403471 non-null  float64
 7   AIR_TIME             1403471 non-null  float64
 8   DISTANCE             1403471 non-null  int64  
 9   TAXI_IN              1403471 non-null  float64
 10  DIVERTED             1403471 non-null  int64  
 11  CANCELLED            1403471 non-null  int64  
 12  AIR_SYSTEM_DELAY     1403471 non-null  float64
 13  SECURITY_DELAY       1403471 non-null  float64
 14  AIRLINE_DELAY        1403471 non-null  float64
 15

As we see above, there are a few columns that are not numerical, and it is important for these to be transformed before proceeding with modeling. I have decided to use label encoding, since all of these variables are categorical variables.

In [19]:
le = LabelEncoder()

X['AIRLINE_CODE']= le.fit_transform(X['AIRLINE_CODE'])
X['ORIG_AIRPORT'] = le.fit_transform(X['ORIG_AIRPORT'])
X['DEST_AIRPORT'] = le.fit_transform(X['DEST_AIRPORT'])

In [20]:
X = pd.get_dummies(X, columns=['MONTH', 'DAY_OF_WEEK'])

### Splitting into train and test sets:

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=666)

I am also going to scale the data so that all the variables are in a uniform scale. This will let the regressor weight all the columns uniformly.

In [22]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Now that I have finished pre-processing and feature selecting, it is train to choose and train models to predict how long a particular flight will be delayed upon arrival. I am going to test the performance of various models before deciding which model I want to tune.

In [23]:
for model, name in zip([ensemble.RandomForestRegressor(), ensemble.GradientBoostingRegressor(),
                        tree.DecisionTreeRegressor(), LinearRegression()], 
     ['Random Forest Regressor', 'Gradient Boosting Regressor', 'Decision Tree Regressor',
     'Linear Regression']):
    model1 = model.fit(X_train, y_train)
    y_pred = model1.predict(X_test)
    print(name)
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('R2 : ',r2_score(y_test, y_pred))
    print()

Random Forest Regressor
Mean Absolute Error: 0.5711842999035724
Mean Squared Error: 4.300947444672979
Root Mean Squared Error: 2.073872571946738
R2 :  0.9972877439740886

Gradient Boosting Regressor
Mean Absolute Error: 5.861076559749258
Mean Squared Error: 63.629400053702554
Root Mean Squared Error: 7.976803874591788
R2 :  0.9598741379798696

Decision Tree Regressor
Mean Absolute Error: 1.0164757910137232
Mean Squared Error: 7.38459583604486
Root Mean Squared Error: 2.717461285105063
R2 :  0.9953431389681267

Linear Regression
Mean Absolute Error: 5.285331088876385
Mean Squared Error: 60.11562382929442
Root Mean Squared Error: 7.753426586309722
R2 :  0.9620899894546787



We see that the Random Forest and Decision Tree models have the lowest mean absolute error out of all of the models. I am going to tune and train a random forest on my data so as to avoid any chance of overfitting (since decision trees are prone to overfitting).

In [24]:
rf = ensemble.RandomForestRegressor()

I have decided to use randomized search instead of grid search to save some time.

In [25]:
param_grid = {'bootstrap': [True, False], 'max_depth': [5,10,None], 'max_features': ['auto', 'log2'], 'n_estimators': [8,9,10,11,12,13,14,15,16,17,18,19,20]}

In [26]:
rand_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=20, scoring='neg_mean_absolute_error', cv=5, n_jobs=1, verbose=0, return_train_score=True)

In [27]:
rand_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20, n_jobs=1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, None],
                                        'max_features': ['auto', 'log2'],
                                        'n_estimators': [8, 9, 10, 11, 12, 13,
                                                         14, 15, 16, 17, 18, 19,
                                                         20]},
                   return_train_score=True, scoring='neg_mean_absolute_error')

In [28]:
print(rand_search.best_params_)

{'n_estimators': 17, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True}


We see the best parameters above, and I will finally test the best model on my test data.

In [30]:
print(rand_search.best_estimator_.score(X_test, y_test))

0.9961248459164971
